## **Import the TensorFlow library**

This code contains a small script that is used for generating a folder full of images. The idea is to show these images to the OpenMV camera when the training with the OL methos is required. Since the idea is to apply a supervised training is required to have a ground truth forthe computation of the error that the model is doing. This script fills a directory with images of digits and creates a txt file in which all the labels are saved. 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dropout, Dense, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from PIL import Image

import numpy as np
import pandas as pd
import seaborn as sns
import random 
import csv 
import sys
import os
import re
from random import seed

import time
#import os, os.path
import cv2
import glob
from keras import applications
#from keras.applications import vgg19
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score

from numpy.ma.core import size

import torchvision.models as models
from torchvision import transforms
ROOT_PATH = os.path.abspath('')
sys.path.insert(0, ROOT_PATH + '/lib')

# VS Code

# IMPORT AND LOAD MODELS

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from sklearn.cluster import KMeans


import random 
from random import seed

import time


import numpy as np
import matplotlib.pyplot as plt


# Load features and labels
features_saved = np.loadtxt('Models/Original_model/ll_features_10.txt')
labels_features_saved = np.loadtxt('Models/Original_model/ll_labels_features_10.txt').astype(int)


# Load model - CFR
model = keras.models.load_model('Models/Original_model/mnist_cnn.h5') # Frozen model 

# Absolute path is needed to load libraries 
ROOT_PATH = os.path.abspath('')
sys.path.append(ROOT_PATH + '/lib')



# Test import
# from lib import simulation_lib
# from lib.simulation_lib import

from lib import Kmeans_lib
from lib.Kmeans_lib import *
from lib.EvalMetrics import *


# LOAD DATASETS

In [24]:
(data_train, label_train),(data_test, label_test) = mnist.load_data() # Load data
print('The original dataset shapes from MNIST are')
print(f'    Train dataset shape: {data_train.shape}')
print(f'    Test dataset shape:  {data_test.shape}')

The original dataset shapes from MNIST are
    Train dataset shape: (60000, 28, 28)
    Test dataset shape:  (10000, 28, 28)


In [25]:
train_samples = label_train.shape[0]
test_samples  = label_test.shape[0]
img_rows, img_cols = 28, 28

trainLow_samples = np.sum(np.where(label_train < 6, 1, 0))
testLow_samples  = np.sum(np.where(label_test  < 6, 1, 0))

# separate in containers data that is lower and higer than 6
# TRAIN - LOW
data_low_train   = np.zeros([trainLow_samples,28,28])
label_low_train  = np.zeros(trainLow_samples)
#       - HIGH
data_high_train  = np.zeros([train_samples-trainLow_samples,28,28])
label_high_train = np.zeros(train_samples-trainLow_samples)

# TEST - LOW
data_low_test   = np.zeros([testLow_samples,28,28])
label_low_test  = np.zeros(testLow_samples)

#      - HIGH
data_high_test  = np.zeros([test_samples-testLow_samples,28,28])
label_high_test = np.zeros(test_samples-testLow_samples)

j,k = 0,0
for i in range(0,train_samples):  
    if(label_train[i]<6):
        data_low_train[j,:,:] = data_train[i,:,:]
        label_low_train[j]    = label_train[i]
        j+=1
    else:
        data_high_train[k,:,:] = data_train[i,:,:]
        label_high_train[k]    = label_train[i]
        k+=1


j,k = 0,0
for i in range(0,test_samples):
    if(label_test[i]>5):
        data_high_test[k,:,:] = data_test[i,:,:]
        label_high_test[k]    = label_test[i]
        k+=1  
    else:
        data_low_test[j,:,:] = data_test[i,:,:]
        label_low_test[j]    = label_test[i]
        j+=1

# Reshape arrays
data_low_train  = data_low_train.reshape(data_low_train.shape[0], img_rows, img_cols, 1)
data_high_train = data_high_train.reshape(data_high_train.shape[0], img_rows, img_cols, 1)
data_low_test   = data_low_test.reshape(data_low_test.shape[0], img_rows, img_cols, 1)
data_high_test  = data_high_test.reshape(data_high_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
    
print('After the separation of the dataset in groups higer and lower/equal than 6 the datasets are:')
print(f'     Train dataset lower than 6 has shape:  {data_low_train.shape}')
print(f'     Train dataset higher than 6 has shape: {data_high_train.shape}')
print()
print(f'     Test dataset lower than 6 has shape:  {data_low_test.shape}')
print(f'     Test dataset higher than 6 has shape: {data_high_test.shape}')

# Normalize the colors from 0-255 to 0-1
data_low_train  = data_low_train.astype(np.float32) / 255.0
data_high_train = data_high_train.astype(np.float32) / 255.0
data_low_test   = data_low_test.astype(np.float32) / 255.0
data_high_test  = data_high_test.astype(np.float32) / 255.0

After the separation of the dataset in groups higer and lower/equal than 6 the datasets are:
     Train dataset lower than 6 has shape:  (36017, 28, 28, 1)
     Train dataset higher than 6 has shape: (23983, 28, 28, 1)

     Test dataset lower than 6 has shape:  (6031, 28, 28, 1)
     Test dataset higher than 6 has shape: (3969, 28, 28, 1)


# FUNCTIONS

In [26]:
def trainOneEpoch_OL(model, x_train, x_test, y_train, y_test, features_saved, labels_features_saved, batch_size):

    # RINOMINATO FEATURES -> SAVED FEATURES
       
    cntr = 1 #  A COSA SERVE?
    learn_rate = model.l_rate
    
    train_samples = x_train.shape[0]
    test_samples = x_test.shape[0]
    tot_samples = train_samples + test_samples
    
    # Merge train and test arrays
    x_tot = np.concatenate((x_train, x_test))   # Images
    y_tot = np.concatenate((y_train, y_test))   # Labels
    
    n_cluster = 10
    err_tot = 0
    cntr_clus = 0
    reminder = tot_samples%batch_size
    max_iter = int(tot_samples//batch_size)

    features_images = np.zeros(tot_samples)
        
    # FEATURE EXTRACTION
    # Estrae le features per ciascuna immagine utilizzando il Frozen Model
    print('**********************************\n Performing features extraction \n')

    for i in range(0, tot_samples):
        features = model.ML_frozen.predict(x_tot[i].reshape((1,28,28,1)), verbose = False)

        if i == 0:
          features_images = np.copy(features)       
        else:
          features_images = np.concatenate((features_images, features))

    # CLUSTERING
    print('**********************************\n Performing clustering\n')

    # Pseudo-labels
    for i in range(0, max_iter):
      
        # pseudo_label, err = k_mean_clustering(features_images[i*batch_size:i*batch_size+batch_size], features_saved, y_tot[i*batch_size:i*batch_size+batch_size], labels_features, n_cluster, batch_size)
        pseudo_label, err = k_mean_clustering(features_images[i*batch_size:i*batch_size+batch_size], features_saved, y_tot[i*batch_size:i*batch_size+batch_size], labels_features_saved, n_cluster, batch_size)
       
        err_tot += err
        pseudo_label = pseudo_label.astype(int)

        if i == 0:
          pseudo_labels = np.copy(pseudo_label)      
        else:
          pseudo_labels = np.append(pseudo_labels, pseudo_label)

        #print(f"\r    Currently at {np.round(np.round(cntr_clus/tot_samples*batch_size,4)*100,2)}% of dataset", end="")
        #print("\n")
        cntr_clus +=1

        # k_mean_clustering(features_run, features_saved, labels_features_run, labels_features_saved, n_cluster, batch_size):
    
    # Pseudo-labels for last batch
    
    if reminder != 0: 
        pseudo_label, err = k_mean_clustering(features_images[max_iter*batch_size:tot_samples], features, y_tot[max_iter*batch_size:tot_samples], labels_features_saved, n_cluster, reminder)
        err_tot = err_tot + err
        pseudo_labels = np.append(pseudo_labels, pseudo_label)

    # Check pseudo-labels and errors in clustering
    
    #print("Pseudo_labels vector: ")
    #print(pseudo_labels)
    #print("\n")

    print("The error in clustering is: ")
    print(int(err_tot/tot_samples*100), '%')
    print("errors:", err_tot)
    print("tot samples", tot_samples)

    # ONLINE-LEARNING

    print('**********************************\n Performing training with OL\n')

    for i in range(0, tot_samples):

        CheckLabelKnown(model, pseudo_labels[i])
    
        y_true_soft = DigitToSoftmax(pseudo_labels[i], model.label)
               
        # Prediction
        
        y_pred = model.predict(features_images[i,:])
        
        # Backpropagation
        cost = y_pred-y_true_soft
        
        for j in range(0,model.W.shape[0]):

            # Update weights
            dW = np.multiply(cost, features_images[i,j]*learn_rate)
            model.W[j,:] = model.W[j,:]-dW

        # Update biases
        db      = np.multiply(cost, learn_rate)
        model.b = model.b-db
        
        # the next part is only to plot the confusion matrix
        # if the train data is finished still train the model but do not save the results
        if(i>=train_samples):

            y_true_soft = DigitToSoftmax(y_tot[i], model.label)
                   
            # Find the max iter for both true label and prediction
            if(np.amax(y_true_soft) != 0):
                max_i_true = np.argmax(y_true_soft)

            if(np.amax(y_pred) != 0):
                max_i_pred = np.argmax(y_pred)

            # Fill up the confusion matrix
            #for k in range(0,len(model.label)):
            #    if(model.label[max_i_pred] == model.std_label[k]):
            #        p = np.copy(k)
            #    if(model.label[max_i_true] == model.std_label[k]):
            #        t = np.copy(k)
            #
            #model.conf_matr[t,p] += 1  

        # ComputeEvalMetrics(y_pred)

        #print(f"\r    Currently at {np.round(np.round(cntr/tot_samples,4)*100,2)}% of dataset", end="")
        #print("\n")
        #cntr +=1

# TRAIN THE MODEL ON THE DIGITS 6-9 (OL METHOD)

In [27]:
'''Function to compute kmean clustering on the new dataset and the saved features'''
def k_mean_clustering(features_run, features_saved, labels_features_run, labels_features_saved, n_cluster, batch_size):

  # Define initial set of features
  labels_init_list = list(range(0, n_cluster))

  # labels_init_list = list([1, 9, 5, 0])
  # labels_init_list = list([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
  # n_cluster = len(labels_init_list)

  # Extract from the saved features the labels that we need
  features_saved_init = []
  labels_features_saved_init = []
  # Extract features of digits considered in labels_init_list
  for i in range(0, len(features_saved)):
      if labels_features_saved[i] in labels_init_list:
        features_saved_init.append(features_saved[i,:])
        labels_features_saved_init.append(labels_features_saved[i])
  
  # Convert list to nparray
  features = np.array(features_saved_init)
  features = features.astype('float32')
  labels_features = np.array(labels_features_saved_init)  

  # Concateno al vettore delle features iniziali le features della nuova batch da analizzare
  features = np.concatenate((features, features_run))
  labels_features = np.append(labels_features, labels_features_run).astype(int)

  # Repeat until clustering is correct
  while True:
    # KMean Clustering
    k_mean = create_k_mean(features, n_cluster)

    # Find pseudolabels for each new image
    # Pseudolabels are computed by looking at the confusion matrix of the saved dataset (where ground truth is known)
    clusters_features_saved = list(k_mean.labels_[0:len(labels_features_saved_init)])
    labels_features_saved_init = list(labels_features_saved_init)
    cluster_list = list(range(0,n_cluster))
    map_clu2lbl, map_lbl2clu = cluster_to_label(clusters_features_saved, labels_features_saved_init, cluster_list, labels_init_list) 
 
    if len(map_clu2lbl) == n_cluster:
        # Exit the loop
        break

  clusters_features = k_mean.labels_

  # Find pseudo label (labels obtained from the model of each image
  pseudolabels_features = np.zeros(len(clusters_features), dtype=int)

  # Initialize error counter
  err = 0

  print("cluster features", clusters_features)
  print("labels features", labels_features)

  # Compute pseudolabels only for new digits
  for i in range(len(clusters_features) - batch_size, len(clusters_features)):
    pseudolabels_features[i] = map_clu2lbl[clusters_features[i]]

    print(pseudolabels_features[i], labels_features[i], clusters_features[i])
    if pseudolabels_features[i] != labels_features[i]:
      err += 1
  
  print(pseudolabels_features)
  # print(labels_features[len(clusters_features)-batch_size:len(clusters_features)])

  # conf_matrix(clusters_features_saved, labels_features_saved_init, , labels_init_list)
  
  # Evaluation metrics
  # ComputeEvalMetrics(labels_features, pseudolabels_features, cluster_list, labels_init_list)

  return pseudolabels_features, err


############

# Dummy  main for debug
if False:
  n = range(0, 10)
  images = data_train[n,:,:]
  labels_features_run = label_train[n]

  features_run = []
  for img in images:
    features_run.append(Custom_Layer(model).ML_frozen.predict(img.reshape((1,28,28,1)))/1000)

  features_run = np.array(features_run).squeeze(axis=1)

  n_cluster = 5 # ARGOMENTO NON USATO
  # AGGIORNARE PASSANDO LA LISTA DI LABELS DA CLUSTERIZZARE
  batch_size = 10

  kmean = k_mean_clustering(features_run, features_saved, labels_features_run, labels_features_saved, n_cluster, batch_size)

In [28]:
n_samples = 4
digits_train = np.zeros((n_samples,28,28))
digits_test = np.zeros((n_samples,28,28))
label_digits_train = np.zeros(n_samples)
label_digits_test = np.zeros(n_samples)

for i in range(0, n_samples):
  n = random.randint(0,len(data_train)-1)
  digits_train[i,:,:] = data_train[n,:,:]
  label_digits_train[i] = label_train[n]
  m = random.randint(0,len(data_test)-1)
  digits_test[i,:,:] = np.copy(data_test[m,:,:])
  label_digits_test[i] = label_test[m]


digits_train  = digits_train.reshape(digits_train.shape[0], img_rows, img_cols, 1).astype(np.float32) / 255.0
digits_test = digits_test.reshape(digits_test.shape[0], img_rows, img_cols, 1).astype(np.float32) / 255.0
input_shape = (img_rows, img_cols, 1)

print(f'The dataset on which the model will be TRAINED has shape {digits_train.shape}') # da cambiare con il nuovo set
print(f'The dataset on which the model will be TESTED has shape  {digits_test.shape}') # da cambiare con il nuovo set

The dataset on which the model will be TRAINED has shape (4, 28, 28, 1)
The dataset on which the model will be TESTED has shape  (4, 28, 28, 1)


In [29]:
Model_OL = Custom_Layer(model)
Model_OL.title      = 'OL'
Model_OL.filename   = 'OL'
Model_OL.l_rate     = 0.01
Model_OL.batch_size = 8

batch_size = Model_OL.batch_size

import importlib
importlib.reload(Kmeans_lib)

#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

trainOneEpoch_OL(Model_OL, digits_train, digits_test,  label_digits_train,  label_digits_test, features_saved, labels_features_saved, batch_size)
#trainOneEpoch_OL(Model_OL, np.concatenate((data_low_train[:25], data_high_train[:25])), np.concatenate((data_low_test[:25], data_high_test[:25])),  np.concatenate((label_low_train[:25], label_high_train[:25])),  np.concatenate((label_low_test[:25], label_high_test[:25])), features, labels_features, batch_size)

**********************************
 Performing features extraction 

**********************************
 Performing clustering

cluster features [9 1 7 3 1 3 5 7 7 1 5 8 3 5 3 7 3 7 1 4 1 4 6 9 1 9 1 1 3 1 4 5 1 9 3 3 4
 5 5 7 3 1 1 4 1 3 4 7 7 9 1 4 8 9 3 4 1 4 4 1 3 1 7 5 3 9 1 3 4 0 3 3 9 5
 6 7 5 5 5 1 7 1 8 1 1 9 7 1 5 5 1 5 7 8 5 5 2 9 4 9 5 7 9 5 7 7 7 1 9 4 4
 9 4 7 4 9 8 9 7 7 8 0 2 6 0 0 8 2 6 6 8 8 8 8 6 6 0 8 2 0 8 6 8 2 8 2 8 8
 6 8 2 8 6 6 2 8 6 0 6 0 0 0 2 8 0 8 0 8 6 8 0 2 6 6 8 2 6 6 2 0 0 6 6 6 2
 2 2 2 0 6 2 2 2 2 2 2 2 2 2 3 8 1 1 1 1 1 1 1]
labels features [2 1 0 4 1 4 5 0 0 1 5 3 4 5 4 0 4 0 1 3 1 3 4 2 1 2 1 1 4 2 3 5 1 2 4 4 3
 5 5 0 4 1 5 3 4 4 3 0 0 2 1 3 2 2 4 3 1 3 3 1 4 1 0 5 4 2 1 4 3 4 4 4 2 5
 4 0 5 5 5 1 0 1 3 1 1 2 0 2 5 5 1 5 0 3 5 5 5 2 3 2 5 0 2 5 0 0 0 3 2 3 3
 2 3 0 3 2 2 2 0 0 7 9 9 6 9 9 7 9 6 6 7 7 7 7 6 6 9 7 8 9 7 6 7 9 7 9 7 7
 6 7 8 7 6 6 9 7 6 9 6 9 9 9 8 7 9 7 9 7 6 7 9 8 6 6 7 8 6 6 8 9 9 6 6 6 8
 8 8 8 9 6 8 8 8 8 8 8 8 8 8 8 5 5 5 3 4 

In [30]:
feat = features_saved[1,:]

##(features_saved.all)
#max(features_saved)

feat.dtype

np.amin(feat)
np.amax(feat)

feat.shape
feat

array([0.03299344, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.01212227, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.70287365, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.41291738, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.37707794,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.03902941, 0.        , 0.        , 1.51821172, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.38334087, 0.        , 0.        ,
       0.        , 2.09713793, 0.        , 2.00183105, 0.        ,
       0.        , 0.        , 0.        , 0.26016855, 0.        ,
       0.59102857, 0.08200867, 0.        , 0.        , 1.35833216,
       0.        , 0.        , 0.        , 0.        , 0.     

In [31]:
X = digits_train[1,:]
YY = Model_OL.ML_frozen.predict(X.reshape((1,28,28,1)), verbose = False)

# YY2 = np.array(YY as features_saved)

np.amin(YY)
np.amax(YY)

YY.shape

np.set_printoptions(suppress=True)

YY

array([[0.        , 2.153017  , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 2.429204  , 0.        , 0.        ,
        1.1958817 , 0.        , 0.        , 0.54145116, 0.        ,
        1.5774523 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.3326033 , 0.        , 1.4055746 , 0.        ,
        0.        , 0.        , 0.        , 1.2649816 , 0.        ,
        0.        , 0.        , 0.        , 0.6006599 , 0.        ,
        0.        , 0.        , 0.        , 0.        , 4.0809655 ,
        0.        , 2.5217166 , 0.        , 0.        , 0.        ,
        0.        , 3.7751284 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 2.0655189 , 0.        , 0.9458386 ,
        0.        , 0.        , 0.        , 0.  

In [32]:



#from simulation_lib import *

#plot_barChart(Model_OL)
#plot_confMatrix(Model_OL)
#plot_table(Model_OL)

# from lib import simulation_lib as sim_lib
# sim_lib.plot_barChart(Model_OL)
# sim_lib.plot_confMatrix(Model_OL)
# sim_lib.plot_table(Model_OL)